In [2]:
import nbformat # used to read .ipynb notebooks
from nbconvert.preprocessors import ExecutePreprocessor # to execute notebook cells
import json 
import torch # used for neural networks
import torch.nn as nn
import torch.optim as optim

In [11]:
# defining a helper funciton to execute a notebook and return a variable
def run_notebook(notebook_path, input_data, expected_output_name):
    """
    Executes a notebook with 'user_input' injected and returns the specified output variable
    """
    # open and read notebook files
    with open(notebook_path) as f:
        nb = nbformat.read(f, as_version=4) # read notebook in version 4 format

    # injecting user_input in notebook
    nb.cells.insert(0, nbformat.v4.new_code_cell(f"user_input = '''{input_data}'''")) # insert new code cell with user input
    nb.cells.append(nbformat.v4.new_code_cell(f"import json\nprint(json.dumps({expected_output_name}))"))
    # creating an executor to run the notebook
    ep = ExecutePreprocessor(timeout=600, kernel_name='python3') # set timeout and kernel
    # run notebook in a temporary directory context
    try: # execute the notebook within its own directory to handle relative paths
        ep.preprocess(nb, {'metadata': {'path': './'}}) # execute in current directory
    except Exception as e:
        print(f"error in notebook execution: {e}")
        return None
    # final_ns = {} # execute global variables created during execution
    # collect the output variable from the last cell that defines it
    # loop through notebook cells to find the output variable
    for output in nb.cells[-1].get("outputs",[]):
        if output.output_type == "stream" and output.name == "stdout":
            try:
                return json.loads(output.text.strip())
            except Exception as e:
                print("JSON parsing failed:",e)
                return None
    print("no valid output found")
    return None

In [12]:
# function to generate a psychologically alignes AI response
def generate_response(emotion_result, personality_traits):
    # select a tone response based on dominant emotion
    emotion = emotion_result.get("dominanr_emotion", "neutral")
    mental_state = emotion_result.get("mental_state", "Neutral")
    frequent_words = emotion_result.get("frequent_words", [])

    # extracting dominant personality traits
    sorted_traits = sorted(personality_traits.items(), key=lambda x: x[1], reverse=True)
    top_trait, top_score = sorted_traits[0]
    # choose personality specific messages
    personality_responses = {
         "Openness": "You seem to value curiosity, creativity, and open-minded thinking.",
         "Conscientiousness": "You appear organized, responsible, and dependable.",
         "Extraversion": "You're energetic, outgoing, and thrive in social settings.",
         "Agreeableness": "You come across as kind, trusting, and empathetic.",
         "Neuroticism": "You may be feeling emotionally sensitive or stressed.",
         "Assertiveness": "You show confidence and decisiveness in your communication.",
         "Altruism": "You demonstrate a caring and generous attitude toward others.",
         "Impulsivity": "You seem spontaneous and quick to act on feelings.",
         "Cautiousness": "You’re careful, thoughtful, and tend to avoid unnecessary risks.",
         "Self-consciousness": "You may be feeling a bit shy or emotionally reserved.",
         "Adventurousness": "You love taking risks and exploring new experiences.",
         "Emotional Awareness": "You're mindful, intuitive, and in touch with emotions.",
         "Industriousness": "You show determination, persistence, and a strong work ethic."
     }
    # crafting the response
    response = (
    f"i sense you are feeling **{emotion}** right now, which might reflect a **{mental_state}** mental state.\n"
    f"Based on your personality, you show strong **{top_trait}** ({top_score}/1).\n"
    f"{personality_responses.get(top_trait, '')}\n"
    )
    if frequent_words:
        response += f"By the way, you've been talking a lot about: {', '.join(frequent_words)}.\n"
    return response

In [17]:
# entry point of the script
if __name__ == "__main__":
    user_input = input("you: ")
    # run the emotion detection notebook 
    emotion_result = run_notebook("EMTN_DTC.ipynb", user_input, "personality_profile")
    # run the personality detection notebook
    personality_result = run_notebook("PRSN_DTC.ipynb", user_input, "analyze_personality(user_input)")
    # validate outputs
    if emotion_result is None or personality_result is None:
        print("failed to process input. check notebook paths and structure")
    else:
        response = generate_response(emotion_result, personality_result)
        print("\nAI:", response)

you:  i am feeling hungry



AI: i sense you are feeling **neutral** right now, which might reflect a **Neutral** mental state.
Based on your personality, you show strong **Neuroticism** (0.57/1).
You may be feeling emotionally sensitive or stressed.

